In [ ]:
import torch

from data import data_helper
from data.data_helper import available_datasets
from models import model_factory
from optimizer.optimizer_helper import get_optim_and_scheduler
from torch.nn import functional as F
from torch import nn

from utils.Logger import Logger

import torchvision
import matplotlib.pyplot as plt

from train_jigsaw import Trainer
from utils import vis

class Container():
    pass

args = Container()
args.batch_size = 128
args.n_classes = 7
args.learning_rate = 0.001
args.epochs = 30
args.network = "caffenet"
args.val_size = 0.1
args.tf_logger = True
args.folder_name = "DANN_grid" # odd_one_out
args.jigsaw_n_classes = 31

args.train_all = True
# args.jig_weight = 0.9
# args.bias_whole_image = 0.8
args.TTA = False
args.suffix = ""

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

source = sorted(["photo", "cartoon", "sketch", "art_painting"])
args.classify_only_sane = True


for args.jig_weight in [0.0]: 
    for args.bias_whole_image in [0.2, 0.5, 0.8]:
        for k, x in enumerate(source):
            args.source = source[:k]+source[k+1:]
            args.target = x
            for i in range(3):
                print("\n%s to %s - %d jigsaw classes, split %d" % ("-".join(args.source), 
                                                                  args.target, 
                                                                  args.jigsaw_n_classes,
                                                                  i))
                trainer = Trainer(args, device)
                trainer.do_training()


cartoon-photo-sketch to art_painting - 31 jigsaw classes, split 0
Using Caffe AlexNet
Dataset size: train 7150, val 793, test 2048
Step size: 24
Saving to /home/enoon/code/2018/JigsawDA/myJigsaw/utils/../logs/DANN_grid/cartoon-photo-sketch_to_art_painting/eps30_bs128_lr0.001_class7_jigClass31_jigWeight0_TAll_bias0.2_classifyOnlySane_534
New epoch - lr: 0.001
0/55 of epoch 1/30 jigsaw : 3.477, class : 2.565, domain : 2.450, lambda : 0.000 - acc jigsaw : 2.34, class : 15.62, domain : 32.03 [bs:128]
Shutting down LAMBDA to prevent implosion
30/55 of epoch 1/30 jigsaw : 3.470, class : 0.845, domain : 0.157, lambda : 0.091 - acc jigsaw : 3.91, class : 53.12, domain : 95.31 [bs:128]
Accuracies on val: jigsaw : 1.13, class : 74.65
Accuracies on test: jigsaw : 0.63, class : 52.59
New epoch - lr: 0.001
0/55 of epoch 2/30 jigsaw : 3.496, class : 0.968, domain : 0.134, lambda : 0.165 - acc jigsaw : 5.47, class : 46.88, domain : 94.53 [bs:128]
30/55 of epoch 2/30 jigsaw : 3.476, class : 0.740, do

30/55 of epoch 18/30 jigsaw : 3.499, class : 0.234, domain : 1.184, lambda : 0.994 - acc jigsaw : 3.12, class : 59.38, domain : 35.94 [bs:128]
Accuracies on val: jigsaw : 2.14, class : 84.24
Accuracies on test: jigsaw : 0.54, class : 52.54
New epoch - lr: 0.001
0/55 of epoch 19/30 jigsaw : 3.515, class : 0.342, domain : 0.992, lambda : 0.995 - acc jigsaw : 0.00, class : 50.00, domain : 51.56 [bs:128]
30/55 of epoch 19/30 jigsaw : 3.452, class : 0.304, domain : 0.834, lambda : 0.996 - acc jigsaw : 5.47, class : 49.22, domain : 59.38 [bs:128]
Accuracies on val: jigsaw : 5.17, class : 82.85
Accuracies on test: jigsaw : 0.78, class : 46.34
New epoch - lr: 0.001
0/55 of epoch 20/30 jigsaw : 3.468, class : 0.949, domain : 0.854, lambda : 0.996 - acc jigsaw : 1.56, class : 52.34, domain : 57.81 [bs:128]
30/55 of epoch 20/30 jigsaw : 3.488, class : 0.278, domain : 0.918, lambda : 0.997 - acc jigsaw : 5.47, class : 58.59, domain : 57.81 [bs:128]
Accuracies on val: jigsaw : 0.63, class : 85.62
A

0/55 of epoch 7/30 jigsaw : 3.474, class : 1.024, domain : 0.748, lambda : 0.762 - acc jigsaw : 3.91, class : 46.88, domain : 71.09 [bs:128]
Shutting down LAMBDA to prevent implosion
Shutting down LAMBDA to prevent implosion
Shutting down LAMBDA to prevent implosion
30/55 of epoch 7/30 jigsaw : 3.478, class : 0.383, domain : 1.051, lambda : 0.797 - acc jigsaw : 3.91, class : 57.81, domain : 56.25 [bs:128]
Accuracies on val: jigsaw : 4.92, class : 80.08
Accuracies on test: jigsaw : 1.61, class : 47.46
New epoch - lr: 0.001
0/55 of epoch 8/30 jigsaw : 3.463, class : 0.508, domain : 0.872, lambda : 0.823 - acc jigsaw : 0.78, class : 53.12, domain : 63.28 [bs:128]
30/55 of epoch 8/30 jigsaw : 3.463, class : 0.632, domain : 0.992, lambda : 0.850 - acc jigsaw : 3.12, class : 42.97, domain : 51.56 [bs:128]
Accuracies on val: jigsaw : 1.77, class : 84.62
Accuracies on test: jigsaw : 0.20, class : 53.71
New epoch - lr: 0.001
0/55 of epoch 9/30 jigsaw : 3.475, class : 0.355, domain : 0.976, lamb

0/55 of epoch 27/30 jigsaw : 3.477, class : 0.214, domain : 0.900, lambda : 1.000 - acc jigsaw : 3.12, class : 57.81, domain : 58.59 [bs:128]
30/55 of epoch 27/30 jigsaw : 3.480, class : 0.282, domain : 0.856, lambda : 1.000 - acc jigsaw : 1.56, class : 59.38, domain : 64.84 [bs:128]
Accuracies on val: jigsaw : 0.63, class : 92.56
Accuracies on test: jigsaw : 0.24, class : 60.94
New epoch - lr: 0.0001
0/55 of epoch 28/30 jigsaw : 3.490, class : 0.176, domain : 0.862, lambda : 1.000 - acc jigsaw : 3.12, class : 67.19, domain : 62.50 [bs:128]
30/55 of epoch 28/30 jigsaw : 3.469, class : 0.154, domain : 0.889, lambda : 1.000 - acc jigsaw : 8.59, class : 58.59, domain : 67.19 [bs:128]
Accuracies on val: jigsaw : 0.76, class : 91.80
Accuracies on test: jigsaw : 0.54, class : 62.89
New epoch - lr: 0.0001
0/55 of epoch 29/30 jigsaw : 3.495, class : 0.195, domain : 0.888, lambda : 1.000 - acc jigsaw : 2.34, class : 75.78, domain : 61.72 [bs:128]
30/55 of epoch 29/30 jigsaw : 3.499, class : 0.3

Accuracies on val: jigsaw : 1.01, class : 85.37
Accuracies on test: jigsaw : 0.34, class : 57.32
New epoch - lr: 0.001
0/55 of epoch 15/30 jigsaw : 3.499, class : 0.426, domain : 0.956, lambda : 0.981 - acc jigsaw : 0.78, class : 61.72, domain : 54.69 [bs:128]
30/55 of epoch 15/30 jigsaw : 3.468, class : 0.462, domain : 0.865, lambda : 0.984 - acc jigsaw : 4.69, class : 49.22, domain : 60.16 [bs:128]
Accuracies on val: jigsaw : 1.77, class : 87.77
Accuracies on test: jigsaw : 1.37, class : 61.91
New epoch - lr: 0.001
0/55 of epoch 16/30 jigsaw : 3.481, class : 0.324, domain : 0.929, lambda : 0.987 - acc jigsaw : 3.91, class : 57.81, domain : 62.50 [bs:128]
30/55 of epoch 16/30 jigsaw : 3.478, class : 0.590, domain : 0.901, lambda : 0.989 - acc jigsaw : 3.91, class : 57.03, domain : 58.59 [bs:128]
Accuracies on val: jigsaw : 1.89, class : 85.37
Accuracies on test: jigsaw : 2.54, class : 55.76
New epoch - lr: 0.001
0/55 of epoch 17/30 jigsaw : 3.472, class : 0.626, domain : 1.032, lambda

...

Accuracies on val: jigsaw : 13.25, class : 88.67
Accuracies on test: jigsaw : 4.67, class : 86.89
New epoch - lr: 0.0001
0/58 of epoch 28/30 jigsaw : 3.402, class : 0.106, domain : 0.728, lambda : 1.000 - acc jigsaw : 10.94, class : 85.16, domain : 61.72 [bs:128]
30/58 of epoch 28/30 jigsaw : 3.334, class : 0.126, domain : 0.659, lambda : 1.000 - acc jigsaw : 14.06, class : 91.41, domain : 70.31 [bs:128]
Accuracies on val: jigsaw : 13.86, class : 88.31
Accuracies on test: jigsaw : 4.49, class : 87.49
New epoch - lr: 0.0001
0/58 of epoch 29/30 jigsaw : 3.363, class : 0.114, domain : 0.746, lambda : 1.000 - acc jigsaw : 11.72, class : 89.06, domain : 68.75 [bs:128]
30/58 of epoch 29/30 jigsaw : 3.357, class : 0.096, domain : 0.742, lambda : 1.000 - acc jigsaw : 10.16, class : 87.50, domain : 68.75 [bs:128]
Accuracies on val: jigsaw : 16.39, class : 89.40
Accuracies on test: jigsaw : 5.21, class : 87.96
New epoch - lr: 0.0001
0/58 of epoch 30/30 jigsaw : 3.382, class : 0.122, domain : 0.8

Shutting down LAMBDA to prevent implosion
Shutting down LAMBDA to prevent implosion
Shutting down LAMBDA to prevent implosion
Shutting down LAMBDA to prevent implosion
Shutting down LAMBDA to prevent implosion
Shutting down LAMBDA to prevent implosion
Shutting down LAMBDA to prevent implosion
Shutting down LAMBDA to prevent implosion
30/58 of epoch 15/30 jigsaw : 3.340, class : 3.282, domain : 4.376, lambda : 0.000 - acc jigsaw : 35.94, class : 21.09, domain : 39.06 [bs:128]
Shutting down LAMBDA to prevent implosion
Accuracies on val: jigsaw : 11.08, class : 82.17
Accuracies on test: jigsaw : 3.77, class : 74.79
New epoch - lr: 0.001
0/58 of epoch 16/30 jigsaw : 3.408, class : 0.420, domain : 0.981, lambda : 0.987 - acc jigsaw : 4.69, class : 82.03, domain : 55.47 [bs:128]
30/58 of epoch 16/30 jigsaw : 3.426, class : 0.479, domain : 0.988, lambda : 0.989 - acc jigsaw : 7.81, class : 73.44, domain : 57.03 [bs:128]
Accuracies on val: jigsaw : 5.42, class : 84.58
Accuracies on test: jigsa

30/42 of epoch 3/30 jigsaw : 3.531, class : 0.258, domain : 0.277, lambda : 0.424 - acc jigsaw : 3.12, class : 82.81, domain : 90.62 [bs:128]
Accuracies on val: jigsaw : 1.16, class : 88.60
Accuracies on test: jigsaw : 0.84, class : 47.47
New epoch - lr: 0.001
0/42 of epoch 4/30 jigsaw : 3.499, class : 0.316, domain : 0.386, lambda : 0.462 - acc jigsaw : 2.34, class : 78.91, domain : 85.16 [bs:128]
30/42 of epoch 4/30 jigsaw : 3.518, class : 0.427, domain : 0.378, lambda : 0.550 - acc jigsaw : 0.00, class : 71.09, domain : 82.03 [bs:128]
Accuracies on val: jigsaw : 1.32, class : 86.28
Accuracies on test: jigsaw : 0.69, class : 49.63
New epoch - lr: 0.001
0/42 of epoch 5/30 jigsaw : 3.477, class : 0.331, domain : 0.353, lambda : 0.583 - acc jigsaw : 3.12, class : 84.38, domain : 84.38 [bs:128]
30/42 of epoch 5/30 jigsaw : 3.528, class : 0.532, domain : 1.095, lambda : 0.656 - acc jigsaw : 1.56, class : 80.47, domain : 58.59 [bs:128]
Accuracies on val: jigsaw : 0.50, class : 84.79
Accura

Accuracies on val: jigsaw : 0.00, class : 89.75
Accuracies on test: jigsaw : 0.23, class : 64.85
New epoch - lr: 0.001
0/42 of epoch 24/30 jigsaw : 3.517, class : 0.115, domain : 0.825, lambda : 0.999 - acc jigsaw : 1.56, class : 89.84, domain : 60.16 [bs:128]
30/42 of epoch 24/30 jigsaw : 3.503, class : 0.169, domain : 0.879, lambda : 0.999 - acc jigsaw : 3.91, class : 85.16, domain : 62.50 [bs:128]
Accuracies on val: jigsaw : 0.99, class : 88.93
Accuracies on test: jigsaw : 0.23, class : 66.91
New epoch - lr: 0.0001
0/42 of epoch 25/30 jigsaw : 3.512, class : 0.091, domain : 0.872, lambda : 0.999 - acc jigsaw : 3.12, class : 92.19, domain : 56.25 [bs:128]
30/42 of epoch 25/30 jigsaw : 3.507, class : 0.032, domain : 0.729, lambda : 0.999 - acc jigsaw : 2.34, class : 91.41, domain : 61.72 [bs:128]
Accuracies on val: jigsaw : 0.66, class : 90.25
Accuracies on test: jigsaw : 0.61, class : 67.22
New epoch - lr: 0.0001
0/42 of epoch 26/30 jigsaw : 3.502, class : 0.058, domain : 0.769, lamb

Accuracies on test: jigsaw : 2.55, class : 63.25
New epoch - lr: 0.001
0/42 of epoch 13/30 jigsaw : 3.335, class : 0.220, domain : 0.728, lambda : 0.964 - acc jigsaw : 7.03, class : 82.03, domain : 66.41 [bs:128]
30/42 of epoch 13/30 jigsaw : 3.354, class : 0.225, domain : 0.845, lambda : 0.972 - acc jigsaw : 4.69, class : 82.81, domain : 60.94 [bs:128]
Accuracies on val: jigsaw : 11.40, class : 88.43
Accuracies on test: jigsaw : 4.99, class : 60.45
New epoch - lr: 0.001
0/42 of epoch 14/30 jigsaw : 3.382, class : 0.149, domain : 0.824, lambda : 0.974 - acc jigsaw : 3.12, class : 88.28, domain : 67.19 [bs:128]
30/42 of epoch 14/30 jigsaw : 3.372, class : 0.233, domain : 0.918, lambda : 0.980 - acc jigsaw : 10.16, class : 89.06, domain : 59.38 [bs:128]
Accuracies on val: jigsaw : 14.88, class : 88.10
Accuracies on test: jigsaw : 5.29, class : 66.56
New epoch - lr: 0.001
0/42 of epoch 15/30 jigsaw : 3.342, class : 0.208, domain : 0.810, lambda : 0.981 - acc jigsaw : 9.38, class : 83.59, 

In [ ]:
source = sorted(["photo", "cartoon", "sketch", "art_painting"])
for args.jig_weight in [0.0]:
    print("\n======================\n%g\n===================" % args.jig_weight)
    for args.bias_whole_image in [1.0]: # [None, 0.01, 0.05, 0.1, 0.3, 0.5]:
        for k, x in enumerate(source):
            args.source = source[:k]+source[k+1:]
            args.target = x
            for i in range(3):
                print("\n%s to %s - %d jigsaw classes, split %d" % ("-".join(args.source), 
                                                                  args.target, 
                                                                  args.jigsaw_n_classes,
                                                                  i))
                trainer = Trainer(args, device)
                trainer.do_training()

In [ ]:
trainer = Trainer(args, device)
logger, model = trainer.do_training()

In [ ]:
%matplotlib notebook
print(100*(logger.val_acc["class"][-1] + logger.val_acc["class"][-2])/2.)
vis.view_training(logger, "%s->%s eps:%d jigweight:%.1f" % ("-".join(args.source),
                                                            args.target,args.epochs, args.jig_weight))

In [ ]:
dataloaders = {"train": data_helper.get_dataloader(args.source, args.jigsaw_n_classes, "train"),
           "val": data_helper.get_dataloader(args.target, args.jigsaw_n_classes, "val")}
dataset_sizes = {"train": len(dataloaders["train"].dataset),
                "val": len(dataloaders["val"].dataset)}
print(dataset_sizes)

In [ ]:
def get_optim_and_scheduler(network, epochs, lr):
    from torch import optim
    optimizer = optim.SGD(network.get_params(lr), weight_decay=.0005, momentum=.9, nesterov=True, lr=lr)
    step_size = int(epochs * .8)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size)
    print(step_size)
    return optimizer, scheduler

model_ft = model_factory.get_network("caffenet")(jigsaw_classes=args.jigsaw_n_classes+1, classes=args.n_classes)
model_ft = model_ft.to(device)
# print(model_ft)

optimizer, scheduler = get_optim_and_scheduler(model_ft, args.epochs, args.learning_rate)

In [ ]:
def do_epoch(model, source, target, optimizer, logger, device):
    criterion = nn.CrossEntropyLoss()
    model.train()
    for it, ((data, jig_l, class_l), d_idx) in enumerate(source):
        data, jig_l, class_l = data.to(device), jig_l.to(device), class_l.to(device)
        
        optimizer.zero_grad()
        
        jigsaw_logit, class_logit = model(data)
        jigsaw_loss = criterion(jigsaw_logit, jig_l)
        class_loss = criterion(class_logit[d_idx!=target_id], class_l[d_idx!=target_id])
        _, cls_pred = class_logit.max(dim=1)
        _, jig_pred = jigsaw_logit.max(dim=1)
        loss = class_loss + jigsaw_loss * jig_weight
        
        loss.backward()
        optimizer.step()

        logger.log(it, len(source), {"jigsaw": jigsaw_loss.item(), "class": class_loss.item()},
                  {"jigsaw": torch.sum(jig_pred == jig_l.data).item(), "class":torch.sum(cls_pred == class_l.data).item()},
                  data.shape[0])
        del loss, class_loss, jigsaw_loss, jigsaw_logit, class_logit

    model.eval()
    with torch.no_grad():
        jigsaw_correct = 0
        class_correct = 0
        total = 0
        for it, ((data, jig_l, class_l), d_idx) in enumerate(target):
            data, jig_l, class_l = data.to(device), jig_l.to(device), class_l.to(device)
            jigsaw_logit, class_logit = model(data)
            _, cls_pred = class_logit.max(dim=1)
            _, jig_pred = jigsaw_logit.max(dim=1)
            class_correct += torch.sum(cls_pred == class_l.data)
            jigsaw_correct += torch.sum(jig_pred == jig_l.data)
            total += data.shape[0]
        logger.log_test({"jigsaw": float(jigsaw_correct) / total,
                         "class": float(class_correct) / total})


def do_training(args, model, source, target, optimizer, scheduler, device):
    logger = Logger(args)
    for k in range(args.epochs):
        scheduler.step()
        logger.new_epoch(scheduler.get_lr())
        do_epoch(model, source, target, optimizer, logger, device)
    return logger, model

In [ ]:
jig_weight = args.jig_weight
logger, model = do_training(args, model_ft, dataloaders["train"], dataloaders["val"], optimizer, scheduler, device)

In [ ]:
%matplotlib notebook
print(100*(logger.val_acc["class"][-1] + logger.val_acc["class"][-2])/2.)
vis.view_training(logger, "%s->%s eps:%d jigweight:%.1f" % (str(args.source),args.target,args.epochs, jig_weight))

In [ ]:
def to_plt(inp):
    import numpy as np
    inp = inp.numpy().transpose((1, 2, 0))
    inp = np.clip(inp, 0, 1)
    return inp

conv1 = model_ft.features[0] # models.alexnet(pretrained=True).features[0] #model_ft.features[0]
tmp = conv1.weight.cpu().data
tmp = torchvision.utils.make_grid(tmp,normalize=True)
plt.imshow(to_plt(tmp))
plt.show()

In [ ]:
vis.view_training(logger, "%s->%s eps:%d jigweight:%.1f" % (str(source),target,epochs, jig_weight))

In [ ]:
fig, ax1 = plt.subplots()
for k,v in logger.losses.items():
    ax1.plot(v, label=k)
    l = len(v)
updates = l / len(logger.val_acc)
print(updates)
plt.legend()
ax2 = ax1.twinx()
ax2.plot(range(0,l,int(updates)), logger.val_acc, label="Test acc", c='g')
plt.legend()
plt.show()

In [ ]:
for e,k in enumerate(range(0,l,int(updates))):
    print(k, logger.val_acc[e])

In [ ]:



# iter_c = iter(train_datasets)

# for x in range(5):
#     tmp = next(iter_c)
#     image = to_plt(tmp[0])
#     plt.imshow(image)
#     plt.show()

In [ ]:
from data.data_helper import get_val_dataloader, get_train_dataloader
from os.path import join, dirname
# from data.JigsawLoader import JigsawTestDataset
import torch
import matplotlib.pyplot as plt
import numpy as np
import torchvision

In [ ]:
# loader = get_val_dataloader("photo",31,batch_size=10,multi=True)
loader, _ = get_train_dataloader(["photo"],31)

In [ ]:
def to_plt(inp):
    inp = inp.numpy().transpose((1, 2, 0))
    return inp

# dataset = JigsawTestDataset("", join('data/txt_lists', 'dslr_train.txt'), patches=False, classes=31)
# test = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True, drop_last=False)
iter_c = iter(loader)
(tmp, v, c, ooo), d = next(iter_c)
print(tmp[:,0].shape)
for x in range(10):
#     image = tmp[0, x]
    image = torchvision.utils.make_grid(tmp[x],1,normalize=True)
    plt.imshow(to_plt(image))
    plt.show()
    print(ooo[x])

# print(tmp[:,0].shape)
# for x in range(tmp.shape[1]):
# #     image = tmp[0, x]
#     image = torchvision.utils.make_grid(tmp[0, x],1,normalize=True)
#     plt.imshow(to_plt(image))
#     plt.show()
#     print(v[0,x])

    
# print(v.max(), v.min())

In [ ]:
res = []
for k in range(10):
    res.append(torch.rand(5, 100))
res = torch.stack(res,0)
res.mean(1).shape

In [ ]:
from data.JigsawLoader import JigsawDataset
from PIL import Image
import torchvision.transforms as transforms

class JigsawTestDataset(JigsawDataset):
    def __init__(self, *args, **xargs):
        super().__init__(*args, **xargs)
        self._augment_tile = transforms.Compose([
#             transforms.RandomCrop(64),
            transforms.Resize((75, 75), Image.BILINEAR),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        
    def __getitem__(self, index):
        framename = self.data_path + '/' + self.names[index]
        img = Image.open(framename).convert('RGB')
        img = self._image_transformer(img)

        w = float(img.size[0]) / self.grid_size
        n_grids = self.grid_size ** 2
        tiles = [None] * n_grids
        for n in range(n_grids):
            y = int(n / self.grid_size)
            x = n % self.grid_size
            tile = img.crop([x * w, y * w, (x + 1) * w, (y + 1) * w])
            tile = self._augment_tile(tile)
            tiles[n] = tile

        data = torch.stack(tiles, 0)
        return self.returnFunc(data), 0, int(self.labels[index])


In [ ]:
dataset = JigsawTestDataset("", join('data/txt_lists', 'dslr_train.txt'), patches=False, classes=31)

In [ ]:
import torch
import random

In [ ]:
true = 0
for x in range(100):
    true += 0.1 > random.random()
true